In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# # Install required libs
# !pip install git+https://github.com/qubvel/segmentation_models.pytorch
# !pip install albumentations==0.4.6

In [3]:
# !sudo rm -rf /content/seg_recyclables
# !git clone https://github.com/finani/seg_recyclables.git

# import sys
# sys.path.append('/content/seg_recyclables')

In [4]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

print('')

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

print('')

import os
import torch
print('Pytorch version: {}'.format(torch.__version__))
print('Is GPU available: {}'.format(torch.cuda.is_available()))
if torch.cuda.is_available():
  print(torch.cuda.get_device_name(0))
  print('The number of GPUs available: {}'.format(torch.cuda.device_count())) # Tesla P100-PCIE-16GB
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print('CPU count: {}'.format(os.cpu_count()))  # 8

cuda = torch.version.cuda
cudnn = torch.backends.cudnn.version()
cudnn_major = cudnn // 1000
cudnn = cudnn % 1000
cudnn_minor = cudnn // 100
cudnn_patch = cudnn % 100
print('Cuda version: {}'.format(cuda)) # 11.1
print('Cudnn version: {}.{}.{}'.format(cudnn_major, cudnn_minor, cudnn_patch)) # 8.0.5


Tue Dec 28 19:37:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| N/A   61C    P8    19W /  N/A |    747MiB /  7982MiB |     24%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Train Binary Segmentation
Classes =  ['Background', 'UNKNOWN', 'General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing']

## Configures

In [5]:
import os
import sys

import torch
import numpy as np

import segmentation_models_pytorch as smp
from torch.optim import lr_scheduler

import Utils
from ModelManager import ModelManager
from DataManager import DataManager, CustomAugmentation
from TrainManager import TrainManager
from InferManager import InferManager
from LossManager import DiceLoss
from LearningRateManager import CustomCosineAnnealingWarmUpRestarts

Utils.fix_random_seed(random_seed=21)

project_dir = '/home/weebee/recyclables/baseline'
dataset_dir = os.path.join(project_dir, 'input')
save_dir = os.path.join(project_dir, 'saved/tm_test')
if not os.path.isdir(dataset_dir):
    sys.exit('check dataset path!!')
if not os.path.isdir(save_dir):
    os.mkdir(save_dir)


## Train each class

In [6]:
for class_name in Utils.get_classes()[1:]:
    class_name = 'Clothing'
    # Set Configures
    # target_classes = Utils.get_classes()
    target_classes = ['Background']
    target_classes.append(class_name)

    config_dict = {
        'project_name': 'test',
        'run_name': '[TM2_Mv2] ' + class_name,
        'network': 'DeepLabV3Plus',
        'encoder': 'resnet101',
        'encoder_weights': 'imagenet',
        'target_classes': target_classes,
        'activation': None,
        'multi_gpu': False,
        'num_epochs': 50,
        'batch_size': 4,
        'learning_rate_0': 1e-4,
        'number_worker': 8,
        'val_every': 1,
        'note': 'test train'
    }

    # Make Model
    model_manager = ModelManager()
    model = model_manager.make_deeplabv3plus_model(
        encoder=config_dict['encoder'],
        encoder_weights=config_dict['encoder_weights'],
        class_number=len(target_classes),
        activation=config_dict['activation'],
        multi_gpu=config_dict['multi_gpu']
    )

    # Load Dataset
    data_manager = DataManager(dataset_path=dataset_dir)
    data_manager.assignDataLoaders(
        batch_size=config_dict['batch_size'],
        shuffle=True,
        number_worker=config_dict['number_worker'],
        drop_last=True,
        # transform=CustomAugmentation.to_tensor_transform(),
        # transform=CustomAugmentation.medium_transform(),
        transform=CustomAugmentation.medium_transform_v2(),
        target_segmentation=True,
        target_classes=target_classes
    )

    # # path of saved best model
    # model_path = os.path.join('/home/weebee/recyclables/baseline/saved/server_11_e50_train_1/', 'best_model_target_' + '_'.join([v.lower() for v in target_classes[1:]]) +'.pt')

    # # load the saved best model
    # checkpoint = torch.load(model_path, map_location=Utils.get_device())
    # state_dict = checkpoint.state_dict()
    # model.load_state_dict(state_dict)

    criterion = smp.utils.losses.CrossEntropyLoss()
    # criterion = smp.utils.losses.BCEWithLogitsLoss()
    # criterion = tgm.losses.DiceLoss()
    # criterion = DiceLoss()
    # criterion = smp.utils.losses.JaccardLoss()

    optimizer = torch.optim.Adam(
        [dict(params=model.parameters(),
                lr=config_dict['learning_rate_0']
                ),
            ])

    lr_scheduler = None
    # lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
    #     optimizer, max_lr=0.01, steps_per_epoch=10, epochs=epochs, anneal_strategy='cos'
    # )
    # lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    #     optimizer, T_0=1, T_mult=2, eta_min=5e-5,
    # )
    # lr_scheduler = CustomCosineAnnealingWarmUpRestarts(
    #     optimizer, T_0=20, T_mult=1, eta_max=0.1,  T_up=2, gamma=0.5
    # )

    # Run Train
    train_manager = TrainManager()
    train_manager.run_train(
        model=model,
        config_dict=config_dict,
        data_loader=data_manager.train_data_loader,
        val_loader=data_manager.val_data_loader,
        criterion=criterion,
        optimizer=optimizer,
        lr_scheduler=lr_scheduler,
        save_dir=save_dir,
        file_name='best_model_target_' + '_'.join([v.lower() for v in target_classes[1:]]) +'.pt',
        # file_name='best_model_1.pt',
        target_only_p=1.0 # -> 1 set = 4 batch_size
    )
    break

loading annotations into memory...
Done (t=2.31s)
creating index...
index created!
loading annotations into memory...


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Done (t=0.75s)
creating index...
index created!

Start training..
num_epochs: 50
save_dir: /home/weebee/recyclables/baseline/saved/tm_test
file_name: best_model_target_clothing.pt
val_every: 1



wandb: Currently logged in as: inhwan-wee (use `wandb login --relogin` to force relogin)


Epoch: 0/50, Time: 2021-12-28 19:37:34, lr: 0.0001: 100%|██████████| 654/654 [00:49<00:00, 13.34it/s, loss=0.357, mean_loss=0.451, train_count=67]


...


Start validation #31


Epoch: 31: 100%|██████████| 163/163 [00:25<00:00,  6.41it/s, mIoU_all=0.6, mIoU_batch=0.341]



Validation #31  Average Loss: 0.2348, mIoU_all: 0.5999
Best performance at epoch or X0th epoch: 31
Save model in /home/weebee/recyclables/baseline/saved/tm_test



Epoch: 31/50, Time: 2021-12-28 20:17:02, lr: 0.0001:  12%|█▏        | 77/654 [00:08<01:04,  8.91it/s, loss=0.218, mean_loss=0.143, train_count=10]


KeyboardInterrupt: 